In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Giả sử bạn đã có bộ dữ liệu huấn luyện và validation
train_data_dir = '/content/drive/MyDrive/BTL_AI/DATA/Train'
validation_data_dir = '/content/drive/MyDrive/BTL_AI/DATA/Validation'

# Tạo ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Tạo mô hình đơn giản
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile mô hình
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Huấn luyện mô hình
model.fit(train_generator, epochs=10, validation_data=validation_generator)

# Lưu mô hình
model.save('/content/model.h5')


Found 317 images belonging to 5 classes.
Found 81 images belonging to 5 classes.
Epoch 1/10
 8/10 [=======================>......] - ETA: 31s - loss: 5.5552 - accuracy: 0.2500

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


10/10 [==============================] - 215s 21s/step - loss: 4.7813 - accuracy: 0.2934 - val_loss: 1.4958 - val_accuracy: 0.5062
Epoch 2/10
10/10 [==============================] - 36s 4s/step - loss: 1.1067 - accuracy: 0.6372 - val_loss: 1.3768 - val_accuracy: 0.5432
Epoch 3/10
10/10 [==============================] - 36s 4s/step - loss: 0.4573 - accuracy: 0.8423 - val_loss: 1.0003 - val_accuracy: 0.6543
Epoch 4/10
10/10 [==============================] - 38s 4s/step - loss: 0.1421 - accuracy: 0.9653 - val_loss: 1.0451 - val_accuracy: 0.6790
Epoch 5/10
10/10 [==============================] - 36s 4s/step - loss: 0.0304 - accuracy: 1.0000 - val_loss: 1.7124 - val_accuracy: 0.6173
Epoch 6/10
10/10 [==============================] - 45s 5s/step - loss: 0.0140 - accuracy: 0.9968 - val_loss: 1.9608 - val_accuracy: 0.6914
Epoch 7/10
10/10 [==============================] - 37s 4s/step - loss: 0.0145 - accuracy: 1.0000 - val_loss: 1.2725 - val_accuracy: 0.7284
Epoch 8/10
10/10 [===========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
!pip install tensorflow gradio


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import gradio as gr
import numpy as np
from PIL import Image

# Tải mô hình đã huấn luyện
model = load_model('/content/model.h5')

# Định nghĩa các lớp (ví dụ: các loại phương tiện giao thông)
class_names = ['Bike', 'Bus', 'Car', 'Motorbike', 'Truck']

# Hàm xử lý đầu vào và dự đoán
def predict_vehicle(image):
    # Chuyển đổi hình ảnh thành mảng numpy và chuẩn hóa
    image = image.resize((224, 224))  # Chỉnh kích thước theo yêu cầu của mô hình
    image = np.array(image) / 255.0  # Chuẩn hóa hình ảnh
    image = np.expand_dims(image, axis=0)  # Thêm trục batch

    # Dự đoán
    predictions = model.predict(image)
    predicted_class = class_names[np.argmax(predictions[0])]

    return predicted_class

# Định nghĩa giao diện Gradio
iface = gr.Interface(
    fn=predict_vehicle,  # Hàm xử lý dự đoán
    inputs=gr.Image(type="pil", label="Upload an image of a vehicle"),  # Đầu vào là hình ảnh
    outputs=gr.Textbox(label="Predicted Class"),  # Đầu ra là văn bản (tên của loại phương tiện)
    title="Vehicle Recognition",  # Tiêu đề của ứng dụng
    description="Upload an image of a vehicle and the model will predict its type.",  # Mô tả ứng dụng
)

# Chạy ứng dụng Gradio
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ba88bff82ee11b8c21.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
